## Multibin

In [1]:
from RHE.util.file_processing import read_annot

In [ ]:
filename = '/Users/nijiayi/RHE_project/data/annot.txt'
annot_matrix, jack_bin = read_annot(filename, 4)

print(jack_bin)

In [135]:
import numpy as np

class Data:
    def __init__(self):
        self.gen = None
        self.index = 0

In [136]:
def bin_to_snp(annot_matrix_chunk): 
    bin_to_snp_indices = []

    for bin_index in range(annot_matrix_chunk.shape[1]):
        snp_indices = np.nonzero(annot_matrix_chunk[:, bin_index])[0]
        bin_to_snp_indices.append(snp_indices.tolist())

    return bin_to_snp_indices

In [137]:
bin_to_snp_map = bin_to_snp(annot_matrix)

In [138]:
def create_allgen(bin_to_snp_map, X):
    allgen = [Data() for _ in range(len(bin_to_snp_map))]

    for i, data in enumerate(allgen):
        data.index = len(bin_to_snp_map[i])
        data.gen = X[:, bin_to_snp_map[i]].T

    return allgen

In [139]:
from bed_reader import open_bed, sample_file

geno_path="/Users/nijiayi/RHE_project/data/small.bed"
bed = open_bed(geno_path)
X = bed.read()


In [140]:
allgen = create_allgen(bin_to_snp_map, X)

In [141]:
print(allgen[0].index)
print(allgen[0].gen)

3
[[1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
  0. 0. 0. 0. 1. 0. 0. 2. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0.
  0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 2. 2. 0. 0. 0. 0.
  1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 2. 1. 0. 0. 0. 0. 0. 1.
  1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0.
  1. 0. 2. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.
  0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.
  0. 1. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0.
  1. 0. 1. 0. 1. 0. 1. 0. 0. 2. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.
  1. 0. 0. 1. 0. 0. 2. 1. 1. 2. 1. 0. 0. 1. 0. 1. 1. 2. 1. 1. 0. 1. 0. 0.
  1. 0. 0. 1. 0. 1. 2. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
  1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 2. 2. 2. 0. 1. 0. 0. 0. 0. 1. 2. 0. 0. 1.
  0. 1. 0

In [142]:
import random

def init_random_z(Nindv, Nz):
    np.random.seed()  
    return np.random.randn(Nindv, Nz) * np.sqrt(3)

In [143]:
def compute_XXz(bin: Data, all_zb, mailman):
    # TODO: mailman == True
    num_snp = bin.index
    gen = bin.gen

    res = gen @ all_zb
    Nz = np.shape(all_zb)[1]

    means = np.mean(gen, axis=1)

    # stds = np.std(gen, axis=1)
    stds = 1 / np.sqrt(means * (1 - 0.5 * means)) # a sample’s value is a binomial random variable with n = 2

    zb_sum = np.sum(all_zb, axis=0)

    zb_sum = np.array(zb_sum).reshape(-1, 1)

    for j in range(num_snp):
        for k in range(Nz):
            res[j, k] = res[j, k] * stds[j]

    inter = np.array(means * stds).reshape(-1, 1)
    resid =  inter @ zb_sum.T
    inter_zb = res - resid

    for k in range(Nz):
        for j in range(num_snp):
            inter_zb[j, k] = inter_zb[j, k] * stds[j]

    new_zb = inter_zb.T
    new_res = new_zb @ gen

    
    new_resid = new_zb @ np.array(means).reshape(-1, 1)
    new_resid = new_resid * np.ones((1, np.shape(gen)[1]))

    temp = new_res - new_resid
    return temp.T

In [146]:
from RHE.util.geno import *

Njack = 4
Nsnp = 100
step_size = Nsnp // Njack
step_size_rem = Nsnp % Njack
mailman = False
Nz = 10
Nindv = 200

def genotype_stream_pass(X):
    for jack_index in range(Njack):
        read_Nsnp = step_size if jack_index < (Njack - 1) else step_size + step_size_rem
        print("num snp per block", read_Nsnp)
        print("Reading block", jack_index)

        if mailman:
            raise NotImplementedError
        
        start = jack_index * step_size
        end = start + read_Nsnp

        annot_matrix_chunk = annot_matrix[start:end, :]
        X_chunk = X[:, start:end]

        # X_chunk_imp = impute_geno(X_chunk)

        bin_to_snp_map = bin_to_snp(annot_matrix_chunk)
        allgen = create_allgen(bin_to_snp_map, X_chunk)

        all_zb = init_random_z(Nindv, Nz)

        for bin in allgen:
            output = compute_XXz(bin, all_zb, mailman)
            print(output.shape)


genotype_stream_pass(X)

num snp per block 25
Reading block 0
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
num snp per block 25
Reading block 1
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
num snp per block 25
Reading block 2
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
num snp per block 25
Reading block 3
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
(200, 10)
